# 学習済みの分散表現（日本語モデル）の利用

- 単語の類似度
- 類似単語の検索
- 単語の類推 (Word Analogy)
- 短い文の類似度

<a href='https://colab.research.google.com/drive/1X3XmmomQZrR-rPGbo_w6yydDbui86tzK'>Word2Vec Application Tutorial</a> をもとに作成

### ソフトウェアのインストール

spaCy, 日本語モデル (ja_core_news_lg) は 6_1_preprocess_jp.ipynb でインストール済み

In [1]:
import spacy
import numpy as np
import seaborn as sns

# 日本語のモデルをロード
nlp = spacy.load('ja_core_news_lg')

### 単語の類似度

In [2]:
# 猫と犬の類似度
cat = nlp.vocab['猫']
dog = nlp.vocab['犬']
cat.similarity(dog)

0.7271089553833008

In [3]:
# 猫と人
cat.similarity(nlp.vocab['人'])

0.327217161655426

### 類似単語の検索

In [4]:
# 検索対象とする単語
def words_to_include(key, exclude_list=[]):
    return nlp.vocab.strings[key] not in exclude_list

# ベクトルに含まれている単語
allwords = [nlp.vocab[key] for key in nlp.vocab.vectors]

# 類似単語の検索
def similar_words(word, n=10):
    word_lexeme = nlp.vocab[word]
    candidates = sorted(allwords, key=lambda w: word_lexeme.similarity(w), reverse=True)
    return [w.text for w in candidates[:n]]

In [ ]:
similar_words('猫')

### 単語の類推 (Word Analogy)

In [ ]:
# コサイン類似度（= 1 - コサイン距離）
from scipy.spatial.distance import cosine
def similarity_cosine(w1, w2):
    return 1 - cosine(w1, w2)

# ベクトルから類似単語の検索
def similar_words_by_vec(vector, exclude_list=[], n=15):
    candidates = sorted(allwords, key=lambda w: similarity_cosine(vector, w.vector),
                        reverse=True)
    return [w.text for w in candidates[:n] if w not in exclude_list]

In [ ]:
# 男にとっての王は、女にとっては何か？
king = nlp.vocab['王']
man = nlp.vocab['男']
woman = nlp.vocab['女']

# 類推したベクトル
analogy = king.vector - man.vector + woman.vector

# 上記の類推結果と queen との類似度
queen = nlp.vocab['女王']
similarity_cosine(analogy, queen.vector)

In [ ]:
# 類推結果に類似する単語
similar_words_by_vec(analogy, ['王', '王様', '男', '女'])

### 短い文の類似度

- 文の各単語ベクトルの平均値の類似度。
- 長文になると値が平均化して、類似度が高くなりやすい。

In [ ]:
s1 = nlp('猫は可愛い。')
s2 = nlp('犬も可愛い。')
s1.similarity(s2)